In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import numpy as np
from Models import MoELSTM
import os
from collections import OrderedDict
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader

from typing import List, Tuple, Optional, Dict
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
import random


In [4]:
import os
import pandas as pd
import numpy as np
from glob import glob
from ast import literal_eval
from sklearn.metrics import mean_squared_error, mean_absolute_error

def compute_metrics(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # NRMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    nrmse = rmse / np.mean(y_true)

    # MASE
    naive_forecast = y_true[:-1]
    true_lagged = y_true[1:]
    naive_mae = mean_absolute_error(true_lagged, naive_forecast)
    model_mae = mean_absolute_error(y_true, y_pred)
    mase = model_mae / (naive_mae + 1e-8)

    return nrmse, mase


def gather_forecast_metrics(metrics_dir="forecast40-50-168"):
    result_records = []

    # Collect all matching CSV files
    metric_files = glob(os.path.join(metrics_dir, "*_metrics_round*.csv"))

    for file_path in metric_files:
        filename = os.path.basename(file_path)
        parts = filename.split("_")

        # Parse model and strategy
        model = parts[0]
        strategy = parts[1]

        df = pd.read_csv(file_path)
        if "y_true" not in df.columns or "y_pred" not in df.columns or "cid" not in df.columns:
            continue  # skip malformed files

        for _, row in df.iterrows():
            cid = int(row["cid"])
            try:
                y_true = literal_eval(row["y_true"])
                y_pred = literal_eval(row["y_pred"])
            except Exception:
                continue  # skip bad data

            if len(y_true) != len(y_pred) or len(y_true) < 2:
                continue  # not valid for MASE

            nrmse, mase = compute_metrics(y_true, y_pred)
            result_records.append({
                "model": model,
                "strategy": strategy,
                "cid": cid,
                "nrmse": nrmse,
                "mase": mase
            })

    return pd.DataFrame(result_records)


def aggregate_metrics(df: pd.DataFrame):
    summary_records = []

    grouped = df.groupby(["model", "strategy"])
    for (model, strategy), group in grouped:
        nrmse_q1 = group["nrmse"].quantile(0.25)
        nrmse_med = group["nrmse"].median()
        nrmse_q3 = group["nrmse"].quantile(0.75)

        mase_q1 = group["mase"].quantile(0.25)
        mase_med = group["mase"].median()
        mase_q3 = group["mase"].quantile(0.75)

        summary_records.append({
            "model": model,
            "strategy": strategy,
            "nrmse_q1": nrmse_q1,
            "nrmse_median": nrmse_med,
            "nrmse_q3": nrmse_q3,
            "mase_q1": mase_q1,
            "mase_median": mase_med,
            "mase_q3": mase_q3
        })

    return pd.DataFrame(summary_records)




In [5]:

all_metrics_df = gather_forecast_metrics("forecast40-50-168")
summary_df = aggregate_metrics(all_metrics_df)
summary_df.to_csv("forecast_summary_metrics_NRMSE.csv", index=False)
print("Saved summary to forecast_summary_metrics_NRMSE.csv")


KeyError: 'model'

In [14]:

import os
import pandas as pd
import numpy as np
from glob import glob
from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

def compute_metrics_mase_nrmse(y_true, y_pred):
    y_true = np.array(y_true, dtype=np.float64)
    y_pred = np.array(y_pred, dtype=np.float64)

    # Replace NaNs with column means
    y_true_mean = np.nanmean(y_true)
    y_pred_mean = np.nanmean(y_pred)
    # print(y_true_mean)
    # print(y_pred_mean)

    y_true = np.where(np.isnan(y_true), y_true_mean, y_true)
    y_pred = np.where(np.isnan(y_pred), y_pred_mean, y_pred)

    # NRMSE
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    nrmse = rmse / (np.mean(y_true) + 1e-8)

    # MASE
    if len(y_true) < 2:
        return np.nan, np.nan  # MASE needs at least 2 points

    naive_forecast = y_true[:-1]
    true_lagged = y_true[1:]
    naive_mae = mean_absolute_error(true_lagged, naive_forecast)
    model_mae = mean_absolute_error(y_true, y_pred)
    mase = model_mae / (naive_mae + 1e-8)

    return nrmse, mase


def evaluate_predictions_for_model_round(model, strategy, round_num, pred_dir="predictions40-50-168"):
    """
    Compute MASE and NRMSE for all prediction CSVs for a given model + strategy and round.
    
    Args:
        model (str): Model name (e.g., "gru")
        strategy (str): Aggregation strategy (e.g., "fedAvg")
        round_num (int): Round number to evaluate
        pred_dir (str): Directory with prediction CSVs
    
    Returns:
        pd.DataFrame: DataFrame with columns [cid, model, strategy, round, nrmse, mase]
    """
    results = []

    pattern = os.path.join(pred_dir, f"*_{model}_{strategy}.csv")
    files = glob(pattern)

    for file in files:
        try:
            df = pd.read_csv(file)
        except Exception as e:
            print(f"Skipping {file}: {e}")
            continue

        df = df[df['round'] == round_num]
        if df.empty or df.shape[0] < 2:
            continue

        y_true = df['true'].values
        y_pred = df['pred'].values

        # Skip files where all values are NaN
        if np.all(np.isnan(y_true)) or np.all(np.isnan(y_pred)):
            print(f"Skipping due to all-NaN: {file}")
            continue

        nrmse, mase = compute_metrics_mase_nrmse(y_true, y_pred)


        nrmse, mase = compute_metrics_mase_nrmse(y_true, y_pred)

        filename = os.path.basename(file)
        cid = int(filename.split("_")[0])

        results.append({
            "cid": cid,
            "model": model,
            "strategy": strategy,
            "round": round_num,
            "nrmse": nrmse,
            "mase": mase
        })

    return pd.DataFrame(results)



In [15]:
STRATEGIES = ["scaffold_diff001","fedProx_diff001","scaffold_diff","fedAvg_diff0","fedProx","fedAvg","scaffold_lr","fedProx_diff" ]#"diff-diff", scaffold_diff,"fedAvg_diff0","fedProx",,"fedAvg_lr","scaffold_lr" "diff_lr2","das11","das2", "fedAvg_lr","fedAvg_diffsample_dhc"
MODELS = [ "gru","lstm"]#,"gru"
ROUND = 50
for strategy in STRATEGIES:
    for model in MODELS:

        df_metrics = evaluate_predictions_for_model_round(model, strategy, ROUND)
        df_metrics.to_csv(f"NRMSE/metrics_{model}_{strategy}_round{ROUND}_NRMSE.csv", index=False)

Skipping due to all-NaN: predictions40-50-168/583_gru_scaffold_diff001.csv
Skipping due to all-NaN: predictions40-50-168/1004_gru_scaffold_diff001.csv
Skipping due to all-NaN: predictions40-50-168/53_gru_scaffold_diff001.csv
Skipping due to all-NaN: predictions40-50-168/583_lstm_scaffold_diff001.csv
Skipping due to all-NaN: predictions40-50-168/53_lstm_scaffold_diff001.csv
Skipping due to all-NaN: predictions40-50-168/1004_lstm_scaffold_diff001.csv
Skipping due to all-NaN: predictions40-50-168/53_gru_fedProx_diff001.csv
Skipping due to all-NaN: predictions40-50-168/1004_gru_fedProx_diff001.csv
Skipping due to all-NaN: predictions40-50-168/583_gru_fedProx_diff001.csv
Skipping due to all-NaN: predictions40-50-168/53_lstm_fedProx_diff001.csv
Skipping due to all-NaN: predictions40-50-168/1004_lstm_fedProx_diff001.csv
Skipping due to all-NaN: predictions40-50-168/583_lstm_fedProx_diff001.csv
Skipping due to all-NaN: predictions40-50-168/1004_gru_scaffold_diff.csv
Skipping due to all-NaN: pr

In [24]:
STRATEGIES = ["fedAvg_diff0","fedProx","fedAvg","fedProx_diff" ]#"diff-diff", scaffold_diff,"fedAvg_diff0","fedProx",,"fedAvg_lr","scaffold_lr" "diff_lr2","das11","das2", "fedAvg_lr","fedAvg_diffsample_dhc"
MODELS = [ "transformer"]#,"gru"
ROUND = 50
for strategy in STRATEGIES:
    for model in MODELS:

        df_metrics = evaluate_predictions_for_model_round(model, strategy, ROUND)
        df_metrics.to_csv(f"NRMSE/metrics_{model}_{strategy}_round{ROUND}_NRMSE.csv", index=False)

Skipping due to all-NaN: predictions40-50-168/53_transformer_fedAvg_diff0.csv
Skipping due to all-NaN: predictions40-50-168/583_transformer_fedAvg_diff0.csv
Skipping due to all-NaN: predictions40-50-168/1004_transformer_fedAvg_diff0.csv
Skipping due to all-NaN: predictions40-50-168/53_transformer_fedProx.csv
Skipping due to all-NaN: predictions40-50-168/583_transformer_fedProx.csv
Skipping due to all-NaN: predictions40-50-168/1004_transformer_fedProx.csv
Skipping due to all-NaN: predictions40-50-168/53_transformer_fedAvg.csv
Skipping due to all-NaN: predictions40-50-168/53_transformer_fedProx_diff.csv


In [6]:
import os
import pandas as pd
from glob import glob

def summarize_nrmse_mase_by_strategy(nrmse_folder="NRMSE", output_csv="NRMSE_summary_by_strategy.csv"):
    """
    Read all NRMSE/MASE metric files and compute mean, Q1, median, Q3 grouped by strategy.

    Args:
        nrmse_folder (str): Path to folder containing metric CSVs.
        output_csv (str): Output file to write strategy-level summary.
    """
    pattern = os.path.join(nrmse_folder, "*.csv")
    files = glob(pattern)

    all_metrics = []

    for file in files:
        try:
            print(file)
            df = pd.read_csv(file)
            if {'nrmse', 'mase', 'strategy'}.issubset(df.columns):
                all_metrics.append(df[['strategy', 'nrmse', 'mase']])
        except Exception as e:
            print(f"Skipping file {file} due to error: {e}")

    if not all_metrics:
        print("No valid files found.")
        return

    full_df = pd.concat(all_metrics, ignore_index=True)

    # Compute summary grouped by strategy
    summary = (
        full_df
        .groupby("strategy")
        .agg({
            "nrmse": ['mean', 'median', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)],
            "mase":  ['mean', 'median', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)]
        })
    )

    # Rename columns
    summary.columns = [
        "nrmse_mean", "nrmse_median", "nrmse_Q1", "nrmse_Q3",
        "mase_mean",  "mase_median",  "mase_Q1",  "mase_Q3"
    ]
    summary.reset_index(inplace=True)

    # Save to CSV
    summary.to_csv(output_csv, index=False)
    print(f"Strategy-level summary saved to {output_csv}")


In [9]:
import os
import pandas as pd
from glob import glob

def summarize_nrmse_mase_by_strategy_model(nrmse_folder="NRMSE", output_csv="NRMSE_summary_by_strategy_model.csv"):
    """
    Read all NRMSE/MASE metric files and compute mean, Q1, median, Q3 grouped by strategy and model.

    Args:
        nrmse_folder (str): Path to folder containing metric CSVs.
        output_csv (str): Output file to write summary.
    """
    pattern = os.path.join(nrmse_folder, "*.csv")
    files = glob(pattern)

    all_metrics = []

    for file in files:
        try:
            print(f"Processing: {file}")
            df = pd.read_csv(file)
            if {'nrmse', 'mase', 'strategy', 'model'}.issubset(df.columns):
                all_metrics.append(df[['strategy', 'model', 'nrmse', 'mase']])
            else:
                print(f"Missing columns in {file}")
        except Exception as e:
            print(f"Skipping file {file} due to error: {e}")

    if not all_metrics:
        print("No valid files found.")
        return

    full_df = pd.concat(all_metrics, ignore_index=True)

    # Compute summary grouped by strategy and model
    summary = (
        full_df
        .groupby(['strategy', 'model'])
        .agg({
            "nrmse": ['mean', 'median', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)],
            "mase":  ['mean', 'median', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)]
        })
    )

    # Rename columns
    summary.columns = [
        "nrmse_mean", "nrmse_median", "nrmse_Q1", "nrmse_Q3",
        "mase_mean",  "mase_median",  "mase_Q1",  "mase_Q3"
    ]
    summary.reset_index(inplace=True)

    # Save to CSV
    summary.to_csv(output_csv, index=False)
    print(f"Summary saved to {output_csv}")


In [10]:
summarize_nrmse_mase_by_strategy_model(nrmse_folder="NRMSE", output_csv="NRMSE_summary_by_strategy2.csv")

Processing: NRMSE/metrics_gru_fedProx_diff001_round50_NRMSE.csv
Processing: NRMSE/metrics_gru_fedAvg_round50_NRMSE.csv
Processing: NRMSE/metrics_gru_fedProx_round50_NRMSE.csv
Processing: NRMSE/metrics_lstm_fedAvg_round50_NRMSE.csv
Processing: NRMSE/metrics_transformer_fedProx_round50_NRMSE.csv
Processing: NRMSE/metrics_gru_fedProx_diff_round50_NRMSE.csv
Processing: NRMSE/metrics_transformer_fedAvg_diff0_round50_NRMSE.csv
Processing: NRMSE/metrics_lstm_scaffold_lr_round50_NRMSE.csv
Processing: NRMSE/metrics_gru_scaffold_diff001_round50_NRMSE.csv
Processing: NRMSE/metrics_gru_scaffold_diff_round50_NRMSE.csv
Processing: NRMSE/metrics_gru_fedAvg_diff0_round50_NRMSE.csv
Processing: NRMSE/metrics_transformer_fedProx_diff_round50_NRMSE.csv
Processing: NRMSE/metrics_lstm_scaffold_diff_round50_NRMSE.csv
Processing: NRMSE/metrics_lstm_fedAvg_diff0_round50_NRMSE.csv
Processing: NRMSE/metrics_lstm_scaffold_diff001_round50_NRMSE.csv
Processing: NRMSE/metrics_lstm_fedProx_diff001_round50_NRMSE.csv
P

In [11]:
df2 = pd.read_csv("NRMSE_summary_by_strategy2.csv")
df2.sort_values("nrmse_median")

,strategy,model,nrmse_mean,nrmse_median,nrmse_Q1,nrmse_Q3,mase_mean,mase_median,mase_Q1,mase_Q3
2,fedAvg,transformer,6741.775221,0.171685,0.118485,0.254836,10110.725852,1.960908,1.459169,2.755756
5,fedAvg_diff0,transformer,16296.644174,0.187027,0.125852,0.262678,20352.253661,1.925699,1.531512,2.684850
11,fedProx_diff,transformer,23485.652067,0.190823,0.137449,0.279569,35063.356229,2.251106,1.725658,3.080058
14,scaffold_diff,gru,14640.813527,0.197158,0.132124,0.283155,17559.123449,2.062420,1.708255,2.678415
18,scaffold_lr,gru,12137.016963,0.197752,0.133012,0.286501,14250.432430,2.131374,1.753620,2.722754
8,fedProx,transformer,19538.314199,0.199247,0.135948,0.275097,24361.188600,2.074771,1.629732,2.784521
16,scaffold_diff001,gru,4709.369615,0.199783,0.133919,0.292410,4888.371002,2.230783,1.817231,2.798005
15,scaffold_diff,lstm,21342.672959,0.202457,0.137355,0.292852,21270.613456,2.205747,1.825197,2.820330
17,scaffold_diff001,lstm,13841.905091,0.203140,0.139067,0.294203,15207.422818,2.238095,1.841617,2.850515
0,fedAvg,gru,28638.561479,0.204236,0.139593,0.297551,31873.995355,2.291272,1.850625,2.966102


In [12]:
df2 = pd.read_csv("NRMSE_summary_by_strategy2.csv")
df2.sort_values("mase_median")

,strategy,model,nrmse_mean,nrmse_median,nrmse_Q1,nrmse_Q3,mase_mean,mase_median,mase_Q1,mase_Q3
5,fedAvg_diff0,transformer,16296.644174,0.187027,0.125852,0.262678,20352.253661,1.925699,1.531512,2.684850
2,fedAvg,transformer,6741.775221,0.171685,0.118485,0.254836,10110.725852,1.960908,1.459169,2.755756
14,scaffold_diff,gru,14640.813527,0.197158,0.132124,0.283155,17559.123449,2.062420,1.708255,2.678415
8,fedProx,transformer,19538.314199,0.199247,0.135948,0.275097,24361.188600,2.074771,1.629732,2.784521
18,scaffold_lr,gru,12137.016963,0.197752,0.133012,0.286501,14250.432430,2.131374,1.753620,2.722754
15,scaffold_diff,lstm,21342.672959,0.202457,0.137355,0.292852,21270.613456,2.205747,1.825197,2.820330
16,scaffold_diff001,gru,4709.369615,0.199783,0.133919,0.292410,4888.371002,2.230783,1.817231,2.798005
19,scaffold_lr,lstm,22128.562019,0.205092,0.138703,0.293280,26481.009483,2.231177,1.832194,2.841610
17,scaffold_diff001,lstm,13841.905091,0.203140,0.139067,0.294203,15207.422818,2.238095,1.841617,2.850515
11,fedProx_diff,transformer,23485.652067,0.190823,0.137449,0.279569,35063.356229,2.251106,1.725658,3.080058


In [13]:
df2 = pd.read_csv("NRMSE_summary_by_strategy2.csv")
df2.sort_values("nrmse_mean")

,strategy,model,nrmse_mean,nrmse_median,nrmse_Q1,nrmse_Q3,mase_mean,mase_median,mase_Q1,mase_Q3
16,scaffold_diff001,gru,4709.369615,0.199783,0.133919,0.292410,4888.371002,2.230783,1.817231,2.798005
2,fedAvg,transformer,6741.775221,0.171685,0.118485,0.254836,10110.725852,1.960908,1.459169,2.755756
18,scaffold_lr,gru,12137.016963,0.197752,0.133012,0.286501,14250.432430,2.131374,1.753620,2.722754
17,scaffold_diff001,lstm,13841.905091,0.203140,0.139067,0.294203,15207.422818,2.238095,1.841617,2.850515
14,scaffold_diff,gru,14640.813527,0.197158,0.132124,0.283155,17559.123449,2.062420,1.708255,2.678415
5,fedAvg_diff0,transformer,16296.644174,0.187027,0.125852,0.262678,20352.253661,1.925699,1.531512,2.684850
8,fedProx,transformer,19538.314199,0.199247,0.135948,0.275097,24361.188600,2.074771,1.629732,2.784521
15,scaffold_diff,lstm,21342.672959,0.202457,0.137355,0.292852,21270.613456,2.205747,1.825197,2.820330
19,scaffold_lr,lstm,22128.562019,0.205092,0.138703,0.293280,26481.009483,2.231177,1.832194,2.841610
6,fedProx,gru,22160.244924,0.213381,0.145962,0.306956,26849.592462,2.471726,1.985990,3.117479


In [14]:
df2 = pd.read_csv("NRMSE_summary_by_strategy2.csv")
df2.sort_values("mase_mean")

,strategy,model,nrmse_mean,nrmse_median,nrmse_Q1,nrmse_Q3,mase_mean,mase_median,mase_Q1,mase_Q3
16,scaffold_diff001,gru,4709.369615,0.199783,0.133919,0.292410,4888.371002,2.230783,1.817231,2.798005
2,fedAvg,transformer,6741.775221,0.171685,0.118485,0.254836,10110.725852,1.960908,1.459169,2.755756
18,scaffold_lr,gru,12137.016963,0.197752,0.133012,0.286501,14250.432430,2.131374,1.753620,2.722754
17,scaffold_diff001,lstm,13841.905091,0.203140,0.139067,0.294203,15207.422818,2.238095,1.841617,2.850515
14,scaffold_diff,gru,14640.813527,0.197158,0.132124,0.283155,17559.123449,2.062420,1.708255,2.678415
5,fedAvg_diff0,transformer,16296.644174,0.187027,0.125852,0.262678,20352.253661,1.925699,1.531512,2.684850
15,scaffold_diff,lstm,21342.672959,0.202457,0.137355,0.292852,21270.613456,2.205747,1.825197,2.820330
8,fedProx,transformer,19538.314199,0.199247,0.135948,0.275097,24361.188600,2.074771,1.629732,2.784521
19,scaffold_lr,lstm,22128.562019,0.205092,0.138703,0.293280,26481.009483,2.231177,1.832194,2.841610
6,fedProx,gru,22160.244924,0.213381,0.145962,0.306956,26849.592462,2.471726,1.985990,3.117479
